In [1]:
from ARCDR import readARCDR
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
from pyproj import Transformer
import rasterio
from osgeo import gdal

In [2]:
hdr, adf = readARCDR("./adf01467.lbl")

./adf01467.2
parsing adf
hdr bytes =  501


### <b> Transformation from geographic to projected coordinate system to match the CRS of the Magellan gridded SAR backscatter data </b>

In [3]:
arr = np.zeros((len(adf), 2)).astype(np.float32)
## projection from geographic to cartesian
geo_crs = CRS.from_proj4("+proj=longlat +a=6051800 +b=6051800 +no_defs ")
proj_crs = CRS.from_proj4("+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=6051800 +b=6051800 +units=m +no_defs")
proj = Transformer.from_crs(geo_crs, proj_crs)

In [7]:
for i in range(len(adf)):
    arr[i, :] = proj.transform(adf[i]["ALT_FOOTPRINT_LONGITUDE"], adf[i]["ALT_FOOTPRINT_LATITUDE"])
print(arr)
diff_arr = np.gradient(arr, axis = 0)
print(diff_arr)
theta = np.rad2deg(np.arctan(diff_arr[:,1]/diff_arr[:,0]))
print(theta)

[[-18779196.    5779422. ]
 [-18777638.    5773036.5]
 [-18776086.    5766665. ]
 ...
 [        inf         inf]
 [        inf         inf]
 [        inf         inf]]
[[ 1558.   -6385.5 ]
 [ 1555.   -6378.5 ]
 [ 1547.   -6365.25]
 ...
 [     nan      nan]
 [     nan      nan]
 [     nan      nan]]
[-76.288284 -76.29924  -76.33977  ...        nan        nan        nan]


/tmp/ipykernel_46295/4049425347.py:6: RuntimeWarning: invalid value encountered in divide
  theta = np.rad2deg(np.arctan(diff_arr[:,1]/diff_arr[:,0]))


In [5]:
ds = rasterio.open("/home/iganesh/Venus_Magellan_LeftLook_mosaic_global_225m.tif")
SAR_DN = ds.read(1)
ds.close()

In [8]:
for i in range(2) #(arr.shape[1]):
    x0 = arr[0,i]
    y0 = arr[0,i]
    a = ysize*1000/2
    b = xsize*1000/2
    ymin = y0 - 2*a
    ymax = y0 - 2*a
    xmin = x0 - 2*a
    xmax = x0 - 2*a
    gdal.Warp(destNameOrDestDS = "/home/iganesh/temp.tif", 
          srcDSOrSrcDSTab  = "/home/iganesh/Venus_Magellan_LeftLook_mosaic_global_225m.tif",
          outputBounds     = (xmin, xmax, ymin, ymax),
          cropToCutline    = True,
          copyMetadata     = True)
    A = np.deg2rad(90+theta)

(2, 1574)

In [8]:
print(proj.transform(adf[0]["ALT_FOOTPRINT_LONGITUDE"], adf[0]["ALT_FOOTPRINT_LATITUDE"]))
print(proj.transform(0, 0))

(-18779196.266915087, 5779422.137011182)
(0.0, 0.0)


In [ ]:
print(np.diff(adf["ALT_FOOTPRINT_LONGITUDE"]))
print(np.gradient(adf["ALT_FOOTPRINT_LONGITUDE"]))
for i in range(len(adf)):
    arr[:,i] = proj.transform(adf[i]["ALT_FOOTPRINT_LONGITUDE"], adf[i]["ALT_FOOTPRINT_LATITUDE"])
    x, y = print(adf[i]["ALT_FOOTPRINT_LONGITUDE"]) #, adf[i]["ALT_FOOTPRINT_LATITUDE"])